In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape


(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head()

,artist,song,text
0,Kenny Chesney,I'll Be Home For Christmas,I'm dreaming tonight \r\nOf a place I love \...
1,Hank Snow,I Wish It Was Mine,Somewhere a telephone is ringing right now \r...
2,James Taylor,Hello Old Friend,Paris' rooftops were lovely to see \r\nSwitze...
3,Helloween,Back On The Ground,We can race over the mountains \r\nSail over ...
4,Lauryn Hill,The Rainbow Song,RED and YELLOW and PINK and GREEN! \r\nPURPLE...


In [9]:
df['text'][0]

"I'm dreaming tonight  \r\nOf a place I love  \r\nEven more than I usually do.  \r\nAnd although I know  \r\n  \r\nIt's a long road back  \r\nI promise you  \r\nI'll be home for Christmas.  \r\nYou can count on me.  \r\n  \r\nPlease have snow and mistletoe  \r\nAnd presents under the tree.  \r\nChristmas Eve will find me  \r\nWhere the love-light gleams.  \r\n  \r\nI'll be home for Christmas  \r\nIf only in my dreams.  \r\nOh oh ah I'll be home for Christmas.  \r\nYou can count on me.  \r\n  \r\nPlease have snow and mistletoe  \r\nAnd presents under the tree.  \r\nOh Christmas Eve will find me  \r\nWhere the love-light gleams.  \r\n  \r\nI'll be home for Christmas  \r\nIf only in my dreams.  \r\nIf only yeah  \r\nWell now, if only, only, only, only  \r\n  \r\nIf only in my dreams.  \r\nIf only  \r\nWell, well, well Lord  \r\nIf only oh  \r\nIf only in my dreams.\r\n\r\n"

# text cleaning text processing

In [10]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [13]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [16]:
similarity[0]

array([1.        , 0.07083682, 0.02210442, ..., 0.17157487, 0.00815448,
       0.02053087])

In [17]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [18]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [19]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))